This Code Handles the Database to GUI Formation .

# Import the dependencies

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

# Class definitions for Parking Lot and Cars

In [2]:
class ParkingLot:
    def __init__(self, capacity):
        self.capacity = capacity
        self.spaces = [None] * capacity


class Car:
    def __init__(self, car_type, car_number, car_status):
        self.car_type = car_type
        self.car_number = car_number
        self.car_status = car_status

# Read the CSV file and split the records of three attributes:
# Car Type

# Car Number 

# Car Status 
#  1 -> Car is in the parking lot
#  0 -> Car has left the parking lot 


In [3]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    car_inputs = df[['Car Type', 'Number', 'Car Status']].values.tolist()
    return car_inputs


# Algorithm for parking lot allotment

In [4]:
def allocate_parking_space(car_type, car_number, car_status, parking_lots):
    for parking_lot in parking_lots:
        if parking_lot.spaces.count(None) == 0:
            continue  # Move to the next parking lot if the current one is full

        for i, space in enumerate(parking_lot.spaces):
            if space is None:
                if car_type == 'SUV':
                    suv_count = sum(1 for s in parking_lot.spaces if s and s.car_type == 'SUV')
                    if suv_count >= 0.4 * parking_lot.capacity:
                        continue  # SUV limit reached in this parking lot

                if car_type == 'SUV' and any(s and s.car_type == 'SUV' for s in parking_lot.spaces[i - 1:i + 2]):
                    continue  # Don't allow two SUVs to be near

                if car_type in ['Sedan', 'Hatchback']:
                    neighboring_count = 0
                    for j in range(i - 1, i + 2):
                        if 0 <= j < parking_lot.capacity and parking_lot.spaces[j] and parking_lot.spaces[
                            j].car_type in ['Sedan', 'Hatchback']:
                            neighboring_count += 1
                    if neighboring_count >= 2:
                        continue  # Limit reached for Sedans and Hatchbacks

                if car_status == 1:
                    parking_lot.spaces[i] = Car(car_type, car_number, car_status)
                else:
                    parking_lot.spaces[i] = None

                return f'Car {car_number} parked at Lot {parking_lots.index(parking_lot) + 1}, Space {i + 1}'

    return 'No available parking space'

# GUI hard code

In [5]:
def create_parking_gui(parking_lots):
    root = tk.Tk()
    root.title("Parking Lot Status")

    # Color map for car types
    color_map = {'SUV': 'red', 'MidSize-SUV': 'purple', 'Sedan': 'green', 'Hatchback': 'blue'}

    # Create a tkinter frame for the labels
    labels_frame = tk.Frame(root, padx=10, pady=10)
    labels_frame.pack(expand=True, fill=tk.BOTH)

    # Update the labels with parking lot status
    def update_labels():
        for row, parking_lot in enumerate(parking_lots):
            for col, space in enumerate(parking_lot.spaces[:5]):
                if space is None:
                    cell_value = ""
                    cell_color = "white"
                else:
                    cell_value = space.car_number
                    cell_color = color_map[space.car_type]

                label = labels[row][col]
                label.config(text=cell_value, bg=cell_color)
                label.update()

    # Create the labels dynamically
    # Create the labels dynamically
    labels = []
    lot_number = 1  # Initialize the lot number
    for row in range(5):
        row_labels = []
        for col in range(5):
            label = tk.Label(labels_frame, text=str(lot_number), width=25, height=5, relief=tk.RAISED, bd=1,
                         font=("Times New Roman", 12, 'bold'), fg='white')
            label.grid(row=row, column=col, padx=5, pady=5, ipadx=2, ipady=2, sticky="nsew")
            row_labels.append(label)
            #row_labels.append(str(lot_number))
            labels_frame.grid_columnconfigure(col, weight=1)
            #lot_number += 1  # Increment the lot number
        labels.append(row_labels)
        labels_frame.grid_rowconfigure(row, weight=1)

    # Add numbering to the labels
    for i, row_labels in enumerate(labels):
        for j, label in enumerate(row_labels):
            lot_number_label = tk.Label(label, text=str(lot_number), font=("Times New Roman", 12, 'bold'), fg='white')
            lot_number_label.pack()
            lot_number_label.config(fg='black')  # Change the color to black
            lot_number += 1


    update_labels()

    # Create a tkinter frame for the legend
    legend_frame = tk.Frame(root)
    legend_frame.pack(side=tk.RIGHT, fill=tk.X)

    # Create the legend labels
    for car_type, color in color_map.items():
        legend_label = tk.Label(legend_frame, text=car_type, bg=color, fg='white', width=15, padx=5, pady=5,
                                font=("Helvetica", 12))
        legend_label.pack(side=tk.LEFT)

    # Create a tkinter frame for the buttons
    button_frame = tk.Frame(root)
    button_frame.pack(side=tk.BOTTOM, pady=10, fill=tk.X)

    # Add a button to reload the CSV file
    def reload_csv():
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            car_inputs = process_csv(file_path)
            for car_type, car_number, car_status in car_inputs:
                result = allocate_parking_space(car_type, car_number, car_status, parking_lots)
                print(result)
            update_labels()

    reload_button = tk.Button(button_frame, text="Reload CSV", command=reload_csv)
    reload_button.pack(side=tk.LEFT, padx=10)

    # Add a button to close the GUI
    close_button = tk.Button(button_frame, text="Close", command=root.quit)
    close_button.pack(side=tk.LEFT, padx=10)

    root.mainloop()

# Load the CSV file

In [7]:
# Prompt the user to select a CSV file
file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])

if file_path:
    car_inputs = process_csv(file_path)
    parking_lots = [ParkingLot(25) for _ in range(3)]

    for car_type, car_number, car_status in car_inputs:
        for parking_lot in parking_lots:
            for space in parking_lot.spaces:
                if space and space.car_number == car_number:
                    if car_status == 0:
                        parking_lot.spaces.remove(space)
                    break
        else:
            result = allocate_parking_space(car_type, car_number, car_status, parking_lots)
            print(result)

    create_parking_gui(parking_lots)

Car 'MH20 DV 2363' parked at Lot 1, Space 1
Car 'Tin21 BZ 0768' parked at Lot 1, Space 2
Car 'HR5 1V3737' parked at Lot 1, Space 3
Car 'GJi6F 19613' parked at Lot 1, Space 4


-------------------